# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [24]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [25]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [26]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [27]:
!mkdir data

mkdir: data: File exists


In [28]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31314    0 31314    0     0   504k      0 --:--:-- --:--:-- --:--:--  546k


In [29]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70173    0 70173    0     0  1217k      0 --:--:-- --:--:-- --:--:-- 1292k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [30]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [31]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [32]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [33]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [34]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 71)

We can save and load our knowledge graphs as follows.

In [35]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 71)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [36]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [37]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

- SingleHopSpecificQuerySynthesizer: This synthesizer creates straightforward, single-hop queries that require information from a single document. It selects specific chunks from documents, generates keyphrases from them, and formulates queries based on those keyphrases. This approach assesses the system's ability to handle direct information retrieval tasks.

- MultiHopAbstractQuerySynthesizer: This synthesizer generates more complex, multi-hop queries that require synthesizing information from multiple documents. It focuses on creating abstract queries that necessitate reasoning across different pieces of information, evaluating the system's capability to integrate and reason over multiple data points.

- MultiHopSpecificQuerySynthesizer: Similar to the MultiHopAbstractQuerySynthesizer, this synthesizer also creates multi-hop queries. However, it focuses on generating specific queries that require retrieving and combining particular pieces of information from multiple documents. This tests the system's proficiency in handling detailed, multi-step information retrieval tasks.




Finally, we can use our `TestSetGenerator` to generate our testset!

In [38]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How has Meta contributed to the accessibility ...,[Code may be the best application The ethics o...,Meta has significantly contributed to the acce...,single_hop_specifc_query_synthesizer
1,"how python help with LLMs, it good or bad?",[Based Development As a computer scientist and...,Python is beneficial for working with LLMs bec...,single_hop_specifc_query_synthesizer
2,what we know bout GPT-4?,[Simon Willison’s Weblog Subscribe Stuff we fi...,We don’t yet know how to build GPT-4.,single_hop_specifc_query_synthesizer
3,What is the significance of Stable Diffusion i...,[easy to follow. The rest of the document incl...,The context mentions that large language model...,single_hop_specifc_query_synthesizer
4,What advancements did Claude 3 bring to the fi...,[Prompt driven app generation is a commodity a...,"Anthropic’s Claude 3 series launched in March,...",single_hop_specifc_query_synthesizer
5,How has the efficiency and pricing of AI model...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the efficiency and pricing of AI mode...",multi_hop_abstract_query_synthesizer
6,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has seen...,multi_hop_abstract_query_synthesizer
7,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has impr...,multi_hop_abstract_query_synthesizer
8,How has Meta contributed to the development an...,[<1-hop>\n\nCode may be the best application T...,Meta has significantly contributed to the deve...,multi_hop_specific_query_synthesizer
9,How does the concept of AGI relate to the chal...,[<1-hop>\n\nBased Development As a computer sc...,The concept of AGI (Artificial General Intelli...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [39]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [40]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht role did OpenAI play in the development of...,[Code may be the best application The ethics o...,"A year ago, OpenAI was the only organization t...",single_hop_specifc_query_synthesizer
1,How does the concept of AGI relate to the chal...,[Based Development As a computer scientist and...,The concept of AGI (Artificial General Intelli...,single_hop_specifc_query_synthesizer
2,Who is Simon Willison in the context of AI adv...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison is the author of a weblog that ...,single_hop_specifc_query_synthesizer
3,what's up with DALL-E 3 and prompt engineering?,[easy to follow. The rest of the document incl...,One of the top entries this year by amount of ...,single_hop_specifc_query_synthesizer
4,How has the efficiency and pricing of AI model...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the efficiency and pricing of AI mode...",multi_hop_abstract_query_synthesizer
5,How do advancements in large language models a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in large language models (LLMs) h...,multi_hop_abstract_query_synthesizer
6,How do advancements in large language models a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in large language models (LLMs) h...,multi_hop_abstract_query_synthesizer
7,How does the environmental impact of AI relate...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI is a significan...,multi_hop_abstract_query_synthesizer
8,How did the advancements in GPT-4 models in 20...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, the advancements in GPT-4 models sign...",multi_hop_specific_query_synthesizer
9,How has Meta contributed to the advancements i...,[<1-hop>\n\nCode may be the best application T...,Meta has significantly contributed to the adva...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [42]:
from langsmith import Client

if not os.environ.get("LANGCHAIN_API_KEY"):
    raise ValueError("LANGCHAIN_API_KEY not found in environment variables")

client = Client(api_key=os.environ["LANGCHAIN_API_KEY"])

dataset_name = "State of AI Across the Years!!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [43]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [44]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [46]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [47]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [48]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [49]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [50]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [51]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [52]:
rag_chain.invoke({"question" : "What are Agents?"})

'"Agents" is a vague term that refers to AI systems that can act on behalf of users. It encompasses different interpretations, such as systems that perform tasks like a travel agent, and LLMs (large language models) that can utilize tools in a loop to solve problems. Despite excitement around the concept, there are few examples of these agents operating in real-world applications, and the term lacks a clear definition across its various uses.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [53]:
eval_llm = ChatOpenAI(model="gpt-4o-mini")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [54]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [56]:
# import os

evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'glossy-sense-55' at:
https://smith.langchain.com/o/734ffbdf-15d1-57ab-9868-51ec7c04a705/datasets/87476770-5058-4d60-bdf7-d2419c631b71/compare?selectedSessions=74045807-b961-424e-a3dc-2d9be93fbb00




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the training efficiency of Llama 3.1 ...,The training efficiency of Llama 3.1 is signif...,None,"Llama 3.1, with its 405B parameters, required ...",0,1,0,2.724533,e188e8ac-f157-4546-b187-2471a52632e9,98d4f46a-94a7-40fd-9ec7-6eec6208f543
1,How has the development of GPT-4o and its pric...,I don't know.,None,The development of GPT-4o has significantly im...,0,0,1,0.633382,71de90ca-0570-447c-9eb4-0b4c9d944f96,f386ac35-af41-44ec-ae28-fa491d765ed0
2,How has Meta contributed to the advancements i...,Meta has contributed to the advancements in La...,None,Meta has significantly contributed to the adva...,1,0,0,2.927533,ebb115cc-df84-41d5-b3a2-e3cf40241444,527adeb6-a928-49b0-8b35-6cc109624fe8
3,How did the advancements in GPT-4 models in 20...,The advancements in GPT-4 models in 2024 led t...,None,"In 2024, the advancements in GPT-4 models sign...",0,0,1,1.970048,d06f1856-4240-4559-9ba1-1419e6dece14,b5048c7c-7535-4f56-9ae2-db4180f18715
4,How does the environmental impact of AI relate...,The environmental impact of AI is a concern li...,None,The environmental impact of AI is a significan...,1,0,0,2.011910,a9c59d16-3347-4ead-80f6-8436b33a5614,feb0308f-548a-47bf-a4c5-6429b269a27b
5,How do advancements in large language models a...,The context indicates that the environmental i...,None,Advancements in large language models (LLMs) h...,0,0,0,2.230050,6b70e822-9cfa-41bd-af44-d9f1d6d159b8,bccf2a02-2f23-4725-91f6-83ee1f5368ab
6,How do advancements in large language models a...,I don't know.,None,Advancements in large language models (LLMs) h...,0,0,0,1.225111,eb434d4a-19ba-48f5-a6b0-9480cc370c15,055626f5-6b07-4a3f-bdb7-7504fa36fcf8
7,How has the efficiency and pricing of AI model...,"In 2024, the efficiency of AI models has impro...",None,"In 2024, the efficiency and pricing of AI mode...",1,0,0,2.430939,ec1b8559-c67b-418c-ba44-859e08732a60,5c933edb-4dbf-438e-90f8-e038f2489874
8,what's up with DALL-E 3 and prompt engineering?,The context mentions that there was a prompt e...,None,One of the top entries this year by amount of ...,0,0,0,2.363968,9e7c2a87-4913-4744-afb4-fb322b6eb8c2,06dcdeda-cade-4250-98be-5beb5291bf2a
9,Who is Simon Willison in the context of AI adv...,Simon Willison is a blogger who reviewed the a...,None,Simon Willison is the author of a weblog that ...,1,1,0,1.231404,e338503a-c2b7-4fd3-b83a-1063bcae34f3,ad53ce36-132a-4044-b3c5-27b04fa41dfd


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [57]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [58]:
rag_documents = docs

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

- modifying chunk size considerations:
    - retrieval precision
    - contextual coherance
    - system efficency and performance
    - model context window alignment

- Determining an appropriate chunk size is a balancing act. Smaller chunks improve retrieval precision but may lack context, while larger chunks provide more context but can dilute retrieval accuracy and risk exceeding model limitations. 




In [60]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

- Accuracy should improve as a better embedding model is used as they are more likely to capture the nuances of the data and as a result improve retrieval precision.
- There would be higher system latency though, but it probably doesn't matter as ingestion is a background step so users would not be affected (unless you're doing it on the fly).
- Different embedding models are better suited for different types of data and tasks, they can also be fine tuned for specific domains so picking one of these would also make a measurable difference. 

In [61]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [62]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [63]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [64]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents? Oh, they\'re this super vague concept in the AI world, man. Some folks think of them as digital assistants that act on your behalf, like a travel agent doing the heavy lifting. Others see them as LLMs using tools to solve problems in a loop. But here\'s the kicker: nobody really agrees on what "agents" even means. It\'s like a game of charades but nobody\'s on the same page. They\'re still hanging in that "coming soon" limbo, and honestly, it feels like until we get that shiny AGI, we might be waiting a while for these agent dreams to roll out in full force. Yeez!'

Finally, we can evaluate the new chain on the same test set!

In [65]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'upbeat-pull-17' at:
https://smith.langchain.com/o/734ffbdf-15d1-57ab-9868-51ec7c04a705/datasets/87476770-5058-4d60-bdf7-d2419c631b71/compare?selectedSessions=0b037f1e-a38b-461a-a5e2-94159f2b888e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the training efficiency of Llama 3.1 ...,"Yo, check it! DeepSeek v3 is flexin' with a sl...",None,"Llama 3.1, with its 405B parameters, required ...",1,0,1,2.486397,e188e8ac-f157-4546-b187-2471a52632e9,162d99c0-c0a2-45d2-9d83-af8701c73dc7
1,How has the development of GPT-4o and its pric...,"Yo, the drop in pricing for GPT-4o to just $2....",None,The development of GPT-4o has significantly im...,1,1,1,2.207218,71de90ca-0570-447c-9eb4-0b4c9d944f96,f35ef63e-27b7-4081-b622-ecb1672d2170
2,How has Meta contributed to the advancements i...,I don't know.,None,Meta has significantly contributed to the adva...,0,0,0,0.642326,ebb115cc-df84-41d5-b3a2-e3cf40241444,3ce9afab-a599-4b77-b32f-16ce34ec01e3
3,How did the advancements in GPT-4 models in 20...,"Yo, the advancements in GPT-4 models in 2024 w...",None,"In 2024, the advancements in GPT-4 models sign...",0,1,1,3.237501,d06f1856-4240-4559-9ba1-1419e6dece14,76052082-ae93-4351-92c9-f3be91ac8d79
4,How does the environmental impact of AI relate...,"Yo, check it! The environmental impact of AI i...",None,The environmental impact of AI is a significan...,1,0,1,2.436662,a9c59d16-3347-4ead-80f6-8436b33a5614,d7c52e78-8ce5-4838-887d-68e596e0f5ab
5,How do advancements in large language models a...,"Alright, let’s break it down cool style! \n\nA...",None,Advancements in large language models (LLMs) h...,1,0,1,3.579403,6b70e822-9cfa-41bd-af44-d9f1d6d159b8,6fedd0d9-0ad8-4429-89c7-d8d419fe9a26
6,How do advancements in large language models a...,"Yo, the advancements in large language models ...",None,Advancements in large language models (LLMs) h...,1,1,1,4.743330,eb434d4a-19ba-48f5-a6b0-9480cc370c15,e137d938-12b0-4d64-a7d3-449c9d83ba34
7,How has the efficiency and pricing of AI model...,"Yo, in 2024, the efficiency of AI models has l...",None,"In 2024, the efficiency and pricing of AI mode...",1,1,1,3.566026,ec1b8559-c67b-418c-ba44-859e08732a60,42020add-2903-4767-82c5-5fdd4ae7699b
8,what's up with DALL-E 3 and prompt engineering?,"Yo, DALL-E 3 is all about those wild prompt en...",None,One of the top entries this year by amount of ...,0,0,1,1.943046,9e7c2a87-4913-4744-afb4-fb322b6eb8c2,46100f46-2dab-49b7-8150-f88cd7e10b8c
9,Who is Simon Willison in the context of AI adv...,Simon Willison is the dude dishing out insight...,None,Simon Willison is the author of a weblog that ...,1,1,1,2.458091,e338503a-c2b7-4fd3-b83a-1063bcae34f3,cc1b1056-8eea-4f67-b592-9e02a8e63b86


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

# ![Results](results.png)
